In [69]:
import pandas as pd

En primer lugar importamos la base de datos y obtenemos las primeras filas para tener una mejor idea de los datos.

In [115]:
filename='sales_predictions.csv'
df=pd.read_csv(filename)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


A continuación, revisamos la cantidad de filas del dataset, el tipo de datos de cada columna y si hay columnas con valores nulos.

In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


Se puede observar que la base de datos contiene 8523 filas. Hay solo dos columnas que cuentan con valores nulos: Item_Weight que se refiere al peso del producto y Outlet_Size que se refiere a el tamaño de la tienda en cuanto a la superficie total que cubre. Por otro lado, y dada la descripcion de cada variable, los tipos de datos se encuentran correctos por lo que no es necesario revisarlo.

# Análisis de Datos Nulos

## Variable Outlet_Size

En primer lugar analizaremos la variable **Outlet_Size.** 

In [117]:
df['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

La variable **Outlet_Size** es una variable cualitativa con 3 valores solamente: Small, Medium o High.

Dado que se trata de una variable que depende del tamaño de la tienda, podemos revisar dos hipótesis para entender mejor cómo rellenar los valores nulos. 

1. El tamaño de la tienda puede estar relacionado a la venta de cada una
2. El tamaño de la tienda puede estar relacionado a la cantidad de productos que tiene cada tienda

Analizaremos la primera hipótesis, en primer lugar identificamos la cantidad de tiendas distintas que existen, en este caso existen 10 tiendas distintas:

In [118]:
df['Outlet_Identifier'].value_counts()

OUT027    935
OUT013    932
OUT046    930
OUT049    930
OUT035    930
OUT045    929
OUT018    928
OUT017    926
OUT010    555
OUT019    528
Name: Outlet_Identifier, dtype: int64

A continuación, revisaremos si los valores nulos están asociados a solo una tienda o si hay tiendas que tienen tanto valores nulos como no nulos.

In [119]:
df.loc[df['Outlet_Size'].isna(),:]['Outlet_Identifier'].value_counts()

OUT045    929
OUT017    926
OUT010    555
Name: Outlet_Identifier, dtype: int64

Como podemos ver las 3 tiendas que no tienen tamaño son la 045, 017 y 010. Tienen todos sus valores nulos por lo que no tenemos indicio a partir de esa variable el tamaño de tienda que puede tener. Por eso, procederemos a revisar la primera hipótesis:

In [120]:
df.groupby(['Outlet_Identifier','Outlet_Size'])[['Item_Outlet_Sales']].sum()

,,Item_Outlet_Sales
Outlet_Identifier,Outlet_Size,
OUT013,High,2.142664e+06
OUT018,Medium,1.851823e+06
OUT019,Small,1.796941e+05
OUT027,Medium,3.453926e+06
OUT035,Small,2.268123e+06
OUT046,Small,2.118395e+06
OUT049,Medium,2.183970e+06


Podemos ver que no hay una correlación directa entre el tamaño de tienda y la venta total, pues hay tiendas Small que superan las ventas de la única tienda High.

Procedemos a revisar la segunda hipótesis.

In [121]:
df.groupby(['Outlet_Identifier','Outlet_Size'])[['Item_Identifier']].count()

,,Item_Identifier
Outlet_Identifier,Outlet_Size,
OUT013,High,932
OUT018,Medium,928
OUT019,Small,528
OUT027,Medium,935
OUT035,Small,930
OUT046,Small,930
OUT049,Medium,930


In [122]:
df.groupby(['Outlet_Identifier'])[['Item_Identifier']].count()

,Item_Identifier
Outlet_Identifier,
OUT010,555
OUT013,932
OUT017,926
OUT018,928
OUT019,528
OUT027,935
OUT035,930
OUT045,929
OUT046,930


Con lo anterior, solo podriamos definir que la tienda OUT010 es Small dado que la cantidad de productos es muy baja, lo que se asocia a la tienda OUT019 que es Small. Por lo que rellenaremos todas las filas de la tienda OUT010 con Small. En el caso del resto, dejaremos los valores nulos por ahora hasta entender si esta variable tiene una correlación importante o no con las ventas.

In [123]:
dfaux=df.loc[df['Outlet_Identifier']=='OUT010',:]
for i in dfaux.index:
    df['Outlet_Size'][i]='Small'
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6668 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


<ipython-input-123-65200f4cfef5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Outlet_Size'][i]='Small'


## Variable Item_Weight

Ahora revisaremos la variable **Item_Weight**. En primer lugar revisaremos cuantos productos distintos tenemos.

In [108]:
df['Item_Identifier'].value_counts()

FDW13    10
FDG33    10
FDF52     9
NCQ06     9
NCL31     9
         ..
FDO33     1
FDT35     1
DRF48     1
FDY43     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64

Podemos identificar 1559 productos diferentes en la base de datos. Revisaremos a continuación qué productos son los que tienen valores nulos en Item_Weight:

In [109]:
df.loc[df['Item_Weight'].isna(),:]['Item_Identifier'].value_counts()

FDA10    2
FDJ56    2
NCO54    2
FDA55    2
FDU01    2
        ..
FDA11    1
FDQ04    1
FDK10    1
FDQ03    1
FDO34    1
Name: Item_Identifier, Length: 1142, dtype: int64

A partir de lo anterior podemos ver que 1142 productos de los 1559 contienen valores nulos en Item_Weight. A continuación revisaremos si los productos que tienen valores nulos tienen valores no nulos.

In [110]:
#la siguiente lista contiene todos los productos que tienen valores nulos en la base de datos
productosnulos=df.loc[df['Item_Weight'].isna(),:]['Item_Identifier'].to_numpy().tolist()

In [111]:
productossinnulos=[]
for i in productosnulos:
    df2=df.loc[df['Item_Identifier']==i,:]
    df3=df2.loc[df['Item_Weight'].isna(),:]
    if len(df2)-len(df3) != 0 and (i not in productossinnulos): 
        productossinnulos.append(i)
len(productossinnulos)

1138

Como podemos ver de los 1142 productos, 1138 productos tienen valores de peso dentro de la base de datos, por lo que a las filas con valores nulos de estos productos serán llenadas con el valor de peso registrado en las filas no nulas. Para esto crearemos una tabla auxiliar que nos de el producto-peso.

In [112]:
aux=df.groupby(['Item_Identifier'])[['Item_Weight']].mean()

A continuación, llenaremos los productos no nulos con el respectivo valor de peso.

In [113]:
df1=df
for i in productossinnulos:
    dfaux=df1.loc[(df1['Item_Identifier']==i),:]
    dfaux=dfaux.loc[df['Item_Weight'].isna(),:]
    for j in dfaux.index:
        df1['Item_Weight'][j]=aux['Item_Weight'][i]
df1.info()

<ipython-input-113-58dbc3757e85>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Item_Weight'][j]=aux['Item_Weight'][i]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8519 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6668 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


Como podemos ver, ahora solo tenemos 4 valores nulos en Item_Weight, de los cuales, no tenemos mayor información para obtener dichos datos, y por tanto no rellenaremos.

In [114]:
df1.loc[df['Item_Weight'].isna(),:]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
927,FDN52,NaN,Regular,0.130933,Frozen Foods,86.9198,OUT027,1985,Medium,Tier 3,Supermarket Type3,1569.9564
1922,FDK57,NaN,Low Fat,0.079904,Snack Foods,120.0440,OUT027,1985,Medium,Tier 3,Supermarket Type3,4434.2280
4187,FDE52,NaN,Regular,0.029742,Dairy,88.9514,OUT027,1985,Medium,Tier 3,Supermarket Type3,3453.5046
5022,FDQ60,NaN,Regular,0.191501,Baking Goods,121.2098,OUT019,1985,Small,Tier 1,Grocery Store,120.5098


# Otros análisis

## Análisis estadísticos de las variables

Realizamos un analisis estadistico inicial de la variables, donde el valor que más llama la atención es que hay productos con Item_Visibility igual a 0, siendo que esta variable se refiere a "El porcentaje de área total de visualización de todos los productos en la tienda asignados a este producto particular".

In [127]:
df1.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,8519.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.875420,0.066132,140.992782,1997.831867,2181.288914
std,4.646098,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.785000,0.026989,93.826500,1987.000000,834.247400
50%,12.650000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


Al revisar cuantas combinaciones item-sala tienen Item_Visibility=0, podemos ver que corresponden a 526 combinaciones.

In [141]:
len(df1.loc[df1['Item_Visibility']==0,:])

526

Lo anterior corresponde al 6,17% de los datos, valor no menor por lo que puede entenderse que no es un error de la data, sino que efectivamente no esta expuesto directamente al público o no es visible para éste.

El resto de variables no presentan valores atípicos que sea necesario de revisar.